In [1]:
import re

In [38]:
corpus_name = 'en_ewt-ud-train.conllu'
with open(corpus_name) as f:
    corpus = []
    #corpus  = re.findall(r"^\d\t(.*)", data)
    for line in f.readlines():
        if re.match(r"^\d\t(.*)", line) != None:
            line = re.split(r'\t', line)
            corpus.append(line)
print('Corpus length (in words) = %d' % len(corpus))

Corpus length (in sentences) = 95250


Пусть первые 7000 слов будут обучающей выборкой, а остальные контрольной

In [85]:
test = corpus[7000:]
corpus = corpus[:7000]

##  HMM-tagger

In [86]:
tags = {}
prob_tags = {}
for token in range(len(corpus)):
    tag = corpus[token][4]
    counter = tags.get(tag)
    if counter == None:
        counter = 0
    tags.update({tag:counter+1})
for tag in tags:
    tag_count = tags.get(tag)
    count = tag_count/len(corpus)
    prob_tags.update({tag:count})

In [105]:
tag_word_bag = {}
word_tag_bag = {}
for token in range(len(corpus)):
    pair = (corpus[token][4], corpus[token][1])
    counter = tag_word_bag.get(pair)
    if counter == None:
        counter = 0
    tag_word_bag.update({pair:counter+1})

for thing in tag_word_bag:
    tag = thing[0]
    word = thing[1]
    count = tag_word_bag.get(thing)/tags.get(tag)
    word_tag_bag.setdefault(word, {})[tag] = count

In [119]:
thing = word_tag_bag.get('killing')

In [132]:
m = []
for i in thing.items():
    l = []
    l.append(i[1])
    l.append(i[0])
    m.append(l)
m = sorted(m)

[[0.0014084507042253522, 'NN'], [0.01652892561983471, 'VBG']]


In [143]:
tagged = []
for token in range(len(test)-1):
    word = test[token][1]
    word_tag = word_tag_bag.get(word)
    if test[token][1] == 1:
        m = []
        for i in word_tag.items():
            l = []
            l.append(i[1])
            l.append(i[0])
        m.append(l)
        m = sorted(m)
        tag = m[0]
        prob = tag[0]* tags.get(tag[1])
    else:
        prev = test[token-1][1]
        prev_tag = word_tag_bag.get(prev)
        m = []
        for i in word_tag.items():
            l = []
            l.append(i[1])
            l.append(i[0])
            m.append(l)
        m = sorted(m)
        tag = m[0]
        m1 = []
        for i in prev_tag.items():
            l = []
            l.append(i[1])
            l.append(i[0])
        m1.append(l)
        m1 = sorted(m)
        tag_prev = m[0]
        prob = tag[0] * tag_prev[0] * tags.get(tag_prev[1])

AttributeError: 'NoneType' object has no attribute 'items'